<a href="https://colab.research.google.com/github/nagi1995/Architectural-Heritage-Elements-image-Dataset/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
import numpy as np
import os
import cv2
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.activations import softmax
from datetime import datetime

In [4]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

'Colab Notebooks'   images      yolov4_bus_car
 downloads	   'My Drive'   yolov4_Dog


In [5]:
train_path = "/mydrive/images/train"
test_path = "/mydrive/images/test"

In [6]:
def return_y_categorical(df):
  df.rename(columns = {0 : "y"}, inplace = True)
  df["y"] = df["y"].astype("category").cat.codes
  return df

# **Color images** 

In [7]:
def return_x_y(path):
  x = []
  y = []
  for img_name in os.listdir(path):
    #print(img_name)
    img = image.load_img(path + "/" + img_name)
    img = image.img_to_array(img)
    x.append(img/255)
    y.append(img_name.split("_")[0])
  y = return_y_categorical(pd.DataFrame(y))
  return (np.array(x), np.array(y))

In [ ]:
datetime.now()

In [8]:
x_train, y_train = return_x_y(train_path)
x_train.shape, y_train.shape

((6141, 128, 128, 3), (6141, 1))

In [16]:
datetime.now()

datetime.datetime(2021, 1, 26, 5, 12, 41, 225402)

In [26]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5, 5), activation = "relu", input_shape = (128, 128, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (5, 5), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation= softmax))

In [27]:
model.compile(optimizer = "adam", loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [28]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 124, 124, 16)      1216      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 62, 62, 16)        0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 58, 58, 32)        12832     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 29, 29, 32)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 25, 25, 64)       

In [29]:
datetime.now()

datetime.datetime(2021, 1, 26, 5, 16, 13, 914355)

In [36]:
model.fit(x_train, y_train, epochs = 50, shuffle = True)

Epoch 1/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3788 - accuracy: 0.8857
Epoch 2/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3751 - accuracy: 0.8845
Epoch 3/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3745 - accuracy: 0.8852
Epoch 4/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3537 - accuracy: 0.8940
Epoch 5/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3945 - accuracy: 0.8841
Epoch 6/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3624 - accuracy: 0.8951
Epoch 7/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3815 - accuracy: 0.8824
Epoch 8/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3387 - accuracy: 0.9013
Epoch 9/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3503 - accuracy: 0.8963
Epoch 10/50
192/192 [==============================] - 3s 14ms/step - loss: 0.3455 - accura

In [31]:
datetime.now()

datetime.datetime(2021, 1, 26, 5, 16, 41, 454740)

In [38]:
model.save("/mydrive/ann_color.h5")

In [39]:
from tensorflow.keras.models import load_model

In [41]:
model = load_model("/mydrive/ann_color.h5")

In [ ]:
datetime.now()

In [42]:
x_test, y_test = return_x_y(test_path)
x_test.shape, y_test.shape

((4094, 128, 128, 3), (4094, 1))

In [ ]:
datetime.now()

In [43]:
model.evaluate(x_test, y_test)

128/128 [==============================] - 1s 7ms/step - loss: 1.2808 - accuracy: 0.7567


[1.2807539701461792, 0.7567171454429626]